In [212]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')
from stargazer.stargazer import Stargazer, LineLocation


In [213]:
from experimentation import data_preparation
from experimentation import regression_OLS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [219]:
df = regression_OLS(file_location = "uk_data.csv", splits = 2, lags = 2, train_share = 0.9, p_cutoff = 0.05)

In [220]:
print(df[0])
print(df[1])
df[2]

+------------------------------------------------------+
| *** OUT OF SAMPLE PERFORMANCE ACROSS SUB-PERIODS *** |
+-----------------+-----------------+------------------+
|                 |        Sample 1 |         Sample 2 |
+-----------------+-----------------+------------------+
| R2              |         -1.3301 |          -0.6487 |
| Mae             |          0.3605 |           0.4837 |
| Mse             |          0.1646 |           0.4086 |
| Rmse            |          0.4057 |           0.6392 |
| Start Date      |      30/04/2013 |       31/05/2022 |
| End Date        |      28/02/2014 |       31/03/2023 |
+-----------------+-----------------+------------------+
                   y  split_1_y_fitted  split_2_y_fitted
date                                                    
2005-01-31  0.876095               NaN               NaN
2005-02-28  0.000000               NaN               NaN
2005-03-31 -0.372208         -0.340843               NaN
2005-04-30  1.245330          0

In [216]:
max(y_test.index)
df3.plot()

AttributeError: plot

In [17]:
df = create_summary_table(file_location = "uk_data.csv", splits = 1, lags = 2, train_share = 0.9, p_cutoff = 0.05)

/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


AttributeError: 'Summary' object has no attribute 'rsquared_adj'